### Batterieversuch Auswertung für W und Q

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path

# ---------------------------------------------------------
# 1) Vernier-TXT einlesen (mit deutschem Dezimal-Komma)
# ---------------------------------------------------------

def find_data_start(filename):
    """
    Bestimme die Zeilennummer, ab der die eigentlichen Messdaten beginnen.
    Erste Zeile, die mit Ziffer oder '-' beginnt, wird als Datenstart genommen.
    """
    with open(filename, encoding="utf-8") as f:
        for i, line in enumerate(f):
            s = line.strip()
            if not s:
                continue
            if s[0].isdigit() or (s[0] == "-" and len(s) > 1 and s[1].isdigit()):
                return i
    raise RuntimeError("Konnte Datenanfang in TXT nicht finden")


def load_vernier_txt(filename):
    """
    Vernier-Textformat einlesen (wie in Gruppe9B_Akku.txt).
    Spalten: t [s], T [°C], U [V], I [A]
    - Tab-getrennt
    - Dezimal-Komma
    - Strom wird beim Einlesen vorzeicheninvertiert
    """
    filename = Path(filename)
    skiprows = find_data_start(filename)

    df = pd.read_csv(
        filename,
        sep="\t",
        header=None,
        names=["t", "T", "U", "I_raw"],
        skiprows=skiprows,
        engine="python",
        decimal=",",    # << wichtig wegen 19,1234 etc.
    )

    # Strom invertieren (weil Aufnahme invertiert war)
    df["I"] = -df["I_raw"]
    df = df.drop(columns=["I_raw"])

    return df


# ---------------------------------------------------------
# 2) Intervalle aus Excel holen (Akku / Batterie getrennt)
# ---------------------------------------------------------

def parse_section_from_excel(excel_path, sheet_name, section_label):
    """
    Liest aus JakobStinkt.xlsx die Intervalle für eine Sektion:
    section_label = 'Akku Zelle' oder 'Batterie Zelle'

    Die Struktur deiner Excel:
    - eine Zeile mit 'Akku Zelle' / 'Batterie Zelle' in Spalte 0
    - eine Zeile darunter die Kopfzeile: Zkylus, R / Ohm, Start / s, Stopp / s, ...
    - darunter die Zeilen mit Zahlen
    """
    df_raw = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)

    # Header-Zeile finden
    hdr_idxs = df_raw.index[df_raw[0] == section_label].tolist()
    if not hdr_idxs:
        raise ValueError(f"Sektion '{section_label}' nicht in Excel gefunden")
    hdr_idx = hdr_idxs[0]

    header_row = df_raw.iloc[hdr_idx]          # z.B. Akku Zelle, Zkylus, R / Ohm, Start / s, ...
    df_section = df_raw.iloc[hdr_idx + 1:].copy()

    # Nur Zeilen behalten, wo Start / s (Spalte 3) nicht NaN ist (also echte Intervalle)
    df_section = df_section[df_section[3].notna()].copy()

    # Spaltennamen aus der Header-Zeile übernehmen
    df_section.columns = header_row.values

    # Nochmals nur Zeilen mit Start- und Stopp-Zeit
    df_section = df_section[
        df_section["Start / s"].notna() & df_section["Stopp / s"].notna()
    ].copy()

    # Ausgabe-DataFrame in Standardform für die Auswertung
    out = pd.DataFrame({
        "Name": df_section["Zkylus"].astype(str),
        "Typ": df_section["Zkylus"].astype(str),
        "t_start": pd.to_numeric(df_section["Start / s"], errors="coerce"),
        "t_end": pd.to_numeric(df_section["Stopp / s"], errors="coerce"),
    })

    out = out[out["t_start"].notna() & out["t_end"].notna()]
    return out


# ---------------------------------------------------------
# 3) Integration U*I über Intervalle + ΔT
# ---------------------------------------------------------

def integrate_interval(df, t_start, t_end):
    """
    Liefert Integrale und Kenngrößen im Zeitintervall [t_start, t_end].
    - W_J: elektrische Arbeit ∫ U*I dt
    - dT_K: Temperaturänderung
    """
    mask = (df["t"] >= t_start) & (df["t"] <= t_end)
    sub = df.loc[mask].copy()

    if sub.empty:
        return None

    t = sub["t"].to_numpy()
    U = sub["U"].to_numpy()
    I = sub["I"].to_numpy()
    T = sub["T"].to_numpy()

    P = U * I
    W = np.trapz(P, t)   # Trapezregel

    dT = T[-1] - T[0]

    return {
        "t_start_eff": t[0],
        "t_end_eff": t[-1],
        "U_mean": U.mean(),
        "I_mean": I.mean(),
        "T_mean": T.mean(),
        "dT_K": dT,
        "W_J": W,
        "n_points": len(sub),
    }


def evaluate_with_intervals(data_df, intervals_df, c_cal=None):
    """
    data_df: Messdaten (aus load_vernier_txt)
    intervals_df: Spalten [Name, Typ, t_start, t_end]
    c_cal: Kalorimeterkonstante [J/K], optional
    """
    results = []

    for _, row in intervals_df.iterrows():
        name = row["Name"]
        typ = row["Typ"]
        t_start = float(row["t_start"])
        t_end = float(row["t_end"])

        integ = integrate_interval(data_df, t_start, t_end)
        if integ is None:
            continue

        # Wärme über Kalorimeter (falls C_cal gegeben)
        Q_cal = None
        if c_cal is not None:
            Q_cal = c_cal * integ["dT_K"]

        results.append({
            "Name": name,
            "Typ": typ,
            "t_start": t_start,
            "t_end": t_end,
            **integ,
            "Q_cal_J": Q_cal,
        })

    return pd.DataFrame(results)


In [15]:
# Pfade kannst du bei dir natürlich anpassen
excel_path = "JakobStinkt.xlsx"
sheet_name = "Tabelle1"

# Intervalle aus Excel holen
intervals_akku = parse_section_from_excel(excel_path, sheet_name, "Akku Zelle")
intervals_batt = parse_section_from_excel(excel_path, sheet_name, "Batterie Zelle")

display(intervals_akku.head())
display(intervals_batt.head())


,Name,Typ,t_start,t_end
3,Base,Base,0.0,600.0
4,Heiz,Heiz,600.0,800.0
5,Base,Base,800.0,1300.0
6,Entladung,Entladung,1300.0,1900.0
7,Base,Base,1900.0,2500.0


,Name,Typ,t_start,t_end
31,Base,Base,0,600
32,Entladung,Entladung,600,1400
33,Base,Base,1400,2000
34,Entladung,Entladung,2000,2800
35,Base,Base,2800,3200


In [16]:
# Akku
df_akku = load_vernier_txt("Gruppe9B_Akku.txt")
result_akku = evaluate_with_intervals(df_akku, intervals_akku, c_cal=650)  # C_cal nach Bedarf
result_akku


,Name,Typ,t_start,t_end,t_start_eff,t_end_eff,U_mean,I_mean,T_mean,dT_K,W_J,n_points,Q_cal_J
0,Base,Base,0.0,600.0,0.0,600.0,4.176845,-0.001105,19.232317,0.125702,-2.772579,601,81.705976
1,Heiz,Heiz,600.0,800.0,600.0,800.0,4.179094,-0.000479,19.512358,0.456158,-0.399458,201,296.502392
2,Base,Base,800.0,1300.0,800.0,1300.0,4.176967,-0.001395,19.771231,0.048644,-2.912890,501,31.618577
3,Entladung,Entladung,1300.0,1900.0,1300.0,1900.0,3.781338,0.322569,19.838180,0.094302,732.452714,601,61.296078
4,Base,Base,1900.0,2500.0,1900.0,2500.0,3.947364,-0.000718,19.929205,0.072166,-2.321714,601,46.907979
5,Entladung,Entladung,2500.0,3100.0,2500.0,3100.0,3.645568,0.367239,20.018771,0.117761,803.878807,601,76.544475
6,Base,Base,3100.0,3700.0,3100.0,3700.0,3.870936,0.007900,20.124309,0.079446,16.611889,601,51.639955
7,Entladung,Entladung,3700.0,4300.0,3700.0,4300.0,3.565568,0.438498,20.219962,0.138215,938.762428,601,89.839893
8,Base,Base,4300.0,4900.0,4300.0,4900.0,3.843476,0.012583,20.338300,0.069070,25.882145,601,44.895686
9,Entladung,Entladung,4900.0,5300.0,4900.0,5299.5,3.531896,0.503895,20.411009,0.098420,711.569217,401,63.972826
